In [1]:
# Entree 1

import numpy as np
import cv2
import pandas as pd
import os
import json

path = "C:/Users/adeik/MaunaKea/Test/"
patients = {}

# Creer un dict avec tous les paths des images par patient
for i in range(80):
    patients[str(i)] = []
    for file in os.listdir(path):
        if file.endswith("_" + str(i) + ".png"):
            patients[str(i)] += [file]

# creer un json pour recupérer le dict apres
with open('unsorted_patients.txt', 'w') as json_file:
    json.dump(patients, json_file)

targets = pd.read_csv("train_target.csv")
targets['patient'] = targets['image_filename'].map(lambda x: x.partition('_')[-1].partition('_')[-1].partition('.')[0])
targets = targets.sort_values(by = ['patient', 'image_filename']).set_index(keys = ['image_filename'])


In [3]:
keys = list(patients.keys())
for key in keys:
    if (patients[key] == []):
        patients.pop(key)
        
# Entree 19

In [4]:
targets.to_csv('new_train_targets.csv', sep = ',')


In [ ]:
#help(targets)


# Widget d'exploration des images

nécéssite ipywidgets et bqplot

In [6]:
import ipywidgets as widgets
from IPython.display import display

# Patient keys
if(True):
    patient_keys = []
    for key in patients.keys():
        patient_keys += [key]
    
# Widgets
if(True):
    patient_number = widgets.Dropdown(options = patient_keys,
                                      value = '16',
                                      description = 'Patient:',
                                      disabled = False)

    file = open(path + patients[patient_number.value][-1], 'rb')
    image = file.read() 

    patient_image = widgets.Image(value = image,
                                  format = 'png',
                                  width = 300,
                                  height = 300)

    patient_class = widgets.Text(value = '0 : Squamous Epithelium',
                                 disabled = True)
    
    patient_play = widgets.Play(value = 0,
                                min = 0,
                                max = len(patients[patient_number.value]) - 1,
                                interval = 150,
                                step = 1,
                                disabled = False)

    patient_next = widgets.Button(description = 'Next',
                                  disabled = False)

    patient_prev = widgets.Button(description = 'Prev',
                                  disabled = False)

    patient_buttons = widgets.HBox([patient_prev, patient_next])
    
    patient_denoise = widgets.Checkbox(value = False, description = 'Denoise', disabled = False)

# Callbacks
if(True):
    def next_patient(change):
        if (int(patient_number.value) < int(patient_keys[-1])):
            patient_number.value = patient_number.options[patient_number.options.index(patient_number.value) + 1]
            patient_prev.disabled = False
        if(int(patient_number.value) == int(patient_keys[-1])):
            patient_next.disabled = True

    def prev_patient(change):
        if (int(patient_number.value) > int(patient_keys[0])):
            patient_number.value = patient_number.options[patient_number.options.index(patient_number.value) - 1]
            patient_next.disabled = False
        if (int(patient_number.value) == int(patient_keys[0])):
            patient_prev.disabled = True

    def patient_playspeed(change):
        if(patient_speed == 'Slow'):
            patient_play.interval = 500

        if(patient_speed == 'Regular'):
            patient_play.interval = 250

        if(patient_speed == 'Fast'):
            patient_play.interval = 100

    def patient_animate(change):
        patient_play.max = len(patients[patient_number.value]) - 1
        
        # image update
        if(True):
            if (patient_play.value < 10):
                image_path = patients[patient_number.value][patient_play.value].partition('_')[0] + '_' + \
                             str(int(patients[patient_number.value][patient_play.value].partition('_')[-1].partition('_')[0])) + \
                             '_' + patients[patient_number.value][patient_play.value].partition('_')[-1].partition('_')[-1]
            else:
                    image_path = patients[patient_number.value][patient_play.value]
            file = open(path + image_path, 'rb')
            image = file.read() 
            if(patient_denoise.value):
                nparr = np.fromstring(image, np.uint8)
                img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                image = cv2.fastNlMeansDenoising(img_np, h  = 15)
                image = cv2.imencode('.png', image)[1].tostring()
            
            patient_image.value = image
        
        # class update
        if(False):
            if(targets.loc[image_path].class_number == 0):
                patient_class.value = '0 : Squamous Epithelium'

            if(targets.loc[image_path].class_number == 1):
                patient_class.value = '1 : Intestinal Metaplasia'

            if(targets.loc[image_path].class_number == 2):
                patient_class.value = '2 : Gastric Metaplasia'

            if(targets.loc[image_path].class_number == 3):
                patient_class.value = '3 : Dysplasia and Cancer'

# Observes
if(True):   
    patient_play.observe(patient_animate, 'value')
    patient_next.on_click(next_patient)
    patient_prev.on_click(prev_patient)
    
display(patient_image, patient_number,patient_class, patient_play, patient_buttons, patient_denoise)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\t\x00\x00\x02\x07\x08\x02\x00\x00\x00\x9a\nX\xdf…

Dropdown(description='Patient:', index=1, options=('9', '16', '20', '21', '26', '27', '28', '33', '37', '38', …

Text(value='0 : Squamous Epithelium', disabled=True)

Play(value=0, interval=150, max=23)

Checkbox(value=False, description='Denoise')